# La Gran Desconexión: Cuando Críticos y Jugadores No Se Entienden - ETL


## Cargar dataset original

In [1]:
import pandas as pd
import numpy as np
import os
from pathlib import Path

# Configuración de la ruta base
DATA_PATH = "../../data/csv"

# Verificar que la ruta existe
if not os.path.exists(DATA_PATH):
    print(f"Error: La ruta {DATA_PATH} no existe")
else:
    print(f"Ruta encontrada: {DATA_PATH}")

# Leer todos los archivos CSV
print("Cargando archivos...")

try:
    # Tabla principal
    games = pd.read_csv(f"{DATA_PATH}/games.csv", quoting=1, escapechar='\\')
    print(f"games.csv cargado: {len(games):,} filas")
    
    # Tablas dimensionales
    game_genres = pd.read_csv(f"{DATA_PATH}/game_genres.csv", quoting=1, escapechar='\\')
    print(f"game_genres.csv cargado: {len(game_genres):,} filas")
    
    game_developers = pd.read_csv(f"{DATA_PATH}/game_developers.csv", quoting=1, escapechar='\\')
    print(f"game_developers.csv cargado: {len(game_developers):,} filas")
    
    game_platforms = pd.read_csv(f"{DATA_PATH}/game_platforms.csv", quoting=1, escapechar='\\')
    print(f"game_platforms.csv cargado: {len(game_platforms):,} filas")
    
    game_ratings = pd.read_csv(f"{DATA_PATH}/game_ratings.csv", quoting=1, escapechar='\\')
    print(f"game_ratings.csv cargado: {len(game_ratings):,} filas")
    
    game_status = pd.read_csv(f"{DATA_PATH}/game_status.csv", quoting=1, escapechar='\\')
    print(f"game_status.csv cargado: {len(game_status):,} filas")
    
    # Métricas derivadas
    game_derived_metrics = pd.read_csv(f"{DATA_PATH}/game_derived_metrics.csv", quoting=1, escapechar='\\')
    print(f"game_derived_metrics.csv cargado: {len(game_derived_metrics):,} filas")
    
    # Opcionales (cargar solo si existen)
    optional_files = ['game_publishers.csv', 'game_stores.csv', 'game_tags.csv', 'game_metacritic_platforms.csv']
    
    for file in optional_files:
        try:
            df = pd.read_csv(f"{DATA_PATH}/{file}", quoting=1, escapechar='\\')
            globals()[file.replace('.csv', '')] = df
            print(f"{file} cargado: {len(df):,} filas")
        except FileNotFoundError:
            print(f"{file} no encontrado, continuando...")

except Exception as e:
    print(f"Error cargando archivos: {e}")

Ruta encontrada: ../../data/csv
Cargando archivos...


C:\Users\Gabri\AppData\Local\Temp\ipykernel_26772\645215834.py:20: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  games = pd.read_csv(f"{DATA_PATH}/games.csv", quoting=1, escapechar='\\')


games.csv cargado: 885,166 filas
game_genres.csv cargado: 1,043,171 filas
game_developers.csv cargado: 988,390 filas
game_platforms.csv cargado: 1,176,534 filas
game_ratings.csv cargado: 119,508 filas
game_status.csv cargado: 885,166 filas
game_derived_metrics.csv cargado: 885,166 filas
game_publishers.csv cargado: 234,990 filas
game_stores.csv cargado: 889,724 filas
game_tags.csv cargado: 4,370,881 filas
game_metacritic_platforms.csv cargado: 8,118 filas


## EDA inicial

In [2]:
print("RESUMEN DE DATOS PRINCIPALES")
print("=" * 50)

print(f"Total de juegos: {len(games):,}")
print(f"Géneros únicos: {game_genres['genre_name'].nunique()}")
print(f"Desarrolladores únicos: {game_developers['developer_name'].nunique()}")

# Verificar campos clave para discrepancias
print("\nCAMPOS CLAVE PARA ANÁLISIS:")
print(f"Juegos con Metacritic: {games['metacritic'].notna().sum():,}")
print(f"Juegos con Rating: {games['rating'].notna().sum():,}")
print(f"Juegos con ambos: {((games['metacritic'].notna()) & (games['rating'].notna())).sum():,}")

# Verificar rango de fechas
games['released'] = pd.to_datetime(games['released'], errors='coerce')
print(f"Rango temporal: {games['released'].min()} a {games['released'].max()}")

RESUMEN DE DATOS PRINCIPALES
Total de juegos: 885,166
Géneros únicos: 19
Desarrolladores únicos: 434233

CAMPOS CLAVE PARA ANÁLISIS:
Juegos con Metacritic: 7,132
Juegos con Rating: 885,166
Juegos con ambos: 7,132
Rango temporal: 1954-08-25 00:00:00 a 2033-10-03 00:00:00


## Crear CSV para la visualización

In [3]:
print("PREPARANDO DATASET PRINCIPAL...")

# Filtrar juegos con datos necesarios para el análisis
games_clean = games[
    (games['metacritic'].notna()) & 
    (games['rating'].notna()) &  
    (games['rating'] > 0.0) & 
    (games['released'].notna())
].copy()

print(f"Juegos válidos para análisis: {len(games_clean):,}")

# Agregar año
games_clean['year'] = games_clean['released'].dt.year

# Filtrar años razonables (evitar outliers)
games_clean = games_clean[
    (games_clean['year'] >= 1975) & 
    (games_clean['year'] <= 2025)
]

print(f"Juegos tras filtro temporal: {len(games_clean):,}")

# Unir con generos
main_genres = game_genres.groupby('game_id').first().reset_index()
games_with_genre = games_clean.merge(
    main_genres[['game_id', 'genre_name']], 
    left_on='id', 
    right_on='game_id', 
    how='inner'
)

print(f"Juegos con género: {len(games_with_genre):,}")

# Unir con desarrolladores
main_developers = game_developers.groupby('game_id').first().reset_index()
games_full = games_with_genre.merge(
    main_developers[['game_id', 'developer_name']], 
    left_on='id', 
    right_on='game_id', 
    how='left'
)

print(f"\nDataset final: {len(games_full):,}")
print(f"Rango de ratings: {games_full['rating'].min():.2f} - {games_full['rating'].max():.2f}")
print(f"Rango de Metacritic: {games_full['metacritic'].min():.2f} - {games_full['metacritic'].max():.2f}")
print("Dataset principal preparado")

PREPARANDO DATASET PRINCIPAL...
Juegos válidos para análisis: 5,806
Juegos tras filtro temporal: 5,806
Juegos con género: 5,801

Dataset final: 5,801
Rango de ratings: 1.00 - 4.81
Rango de Metacritic: 21.00 - 99.00
Dataset principal preparado


In [4]:
print("CALCULANDO MÉTRICAS DE DISCREPANCIA...")

# Crear copia del dataset para trabajar
df_analysis = games_full.copy()

# Normalizar las escalas para comparar
# Metacritic: 0-100 -> Rating: 1-5, convertir rating a escala 0-100
df_analysis['rating_scaled'] = (df_analysis['rating'] - 1) * 25  # 1-5 -> 0-100

# Calcular discrepancia (positivo = usuarios valoran más que críticos)
df_analysis['discrepancy_score'] = (df_analysis['rating_scaled'] - df_analysis['metacritic']) / 10

print(f"Min: {df_analysis['discrepancy_score'].min():.2f}")
print(f"P10: {df_analysis['discrepancy_score'].quantile(0.1):.2f}")
print(f"P25: {df_analysis['discrepancy_score'].quantile(0.25):.2f}")
print(f"P50: {df_analysis['discrepancy_score'].quantile(0.5):.2f}")
print(f"P75: {df_analysis['discrepancy_score'].quantile(0.75):.2f}")
print(f"P90: {df_analysis['discrepancy_score'].quantile(0.9):.2f}")
print(f"Max: {df_analysis['discrepancy_score'].max():.2f}")

# Unir consensus_score con métricas derivadas
df_analysis = df_analysis.merge(
    game_derived_metrics[['game_id', 'consensus_score']], 
    left_on='id', 
    right_on='game_id', 
    how='left'
)
# Clasificar tipos de discrepancia
def classify_discrepancy(score):
    if score >= 0.25: # Los usuarios valoran algo más que los críticos
        return "hidden_gem"
    elif score <= -4.5:
        return "overhyped"
    elif score >= -1.5 and score <= 0.25: # Rango de consenso (ligeramente desviado ya que es la tendencia natural de los datos)
        return "consensus"
    else: # Entre -4.5 y -1.5
        return "polarizing"

df_analysis['discrepancy_type'] = df_analysis['discrepancy_score'].apply(classify_discrepancy)

# Estadísticas básicas
print(f"\nDataset de análisis: {len(df_analysis):,} juegos")
print(f"Discrepancia promedio: {df_analysis['discrepancy_score'].mean():.2f}")
print(f"Discrepancia std: {df_analysis['discrepancy_score'].std():.2f}")
print("\nDistribución de tipos:")
print(df_analysis['discrepancy_type'].value_counts())

print("Métricas de discrepancia calculadas")

CALCULANDO MÉTRICAS DE DISCREPANCIA...
Min: -8.00
P10: -2.62
P25: -1.70
P50: -0.90
P75: -0.25
P90: 0.38
Max: 4.92

Dataset de análisis: 5,801 juegos
Discrepancia promedio: -1.03
Discrepancia std: 1.26

Distribución de tipos:
discrepancy_type
consensus     3343
polarizing    1682
hidden_gem     716
overhyped       60
Name: count, dtype: int64
Métricas de discrepancia calculadas


## Generar CSVs

### Genre_discrepancies

In [5]:
print("Generando genre_discrepancies.csv...")

genre_stats = df_analysis.groupby('genre_name').agg({
    'metacritic': ['mean', 'std', 'count'],
    'rating': ['mean', 'std'],
    'rating_scaled': 'mean',
    'consensus_score': 'mean',
    'discrepancy_score': ['mean', 'std'],
    'playtime': 'mean'
}).round(2)

# Aplanar columnas multi-nivel
genre_stats.columns = ['_'.join(col).strip() for col in genre_stats.columns]

# Renombrar para claridad
genre_discrepancies = genre_stats.rename(columns={
    'metacritic_mean': 'metacritic_avg',
    'metacritic_std': 'metacritic_std',
    'metacritic_count': 'game_count',
    'rating_mean': 'rating_avg',
    'rating_std': 'rating_std',
    'consensus_score_mean': 'consensus_score_avg',
    'discrepancy_score_mean': 'discrepancy_score',
    'discrepancy_score_std': 'discrepancy_std',
    'playtime_mean': 'avg_playtime'
}).reset_index()

# Filtrar géneros con al menos 0 juegos para que sea significativo (quiero también los nichos)
genre_discrepancies = genre_discrepancies[genre_discrepancies['game_count'] >= 0]

# Ordenar por discrepancia
genre_discrepancies = genre_discrepancies.sort_values('discrepancy_score', ascending=False)

# Guardar
genre_discrepancies.to_csv('../../data/processed/genre_discrepancies.csv', index=False)
print(f"genre_discrepancies.csv guardado: {len(genre_discrepancies)} géneros")
print("Top 5 géneros más valorados por usuarios:")
print(genre_discrepancies[['genre_name', 'discrepancy_score', 'game_count']].head())

Generando genre_discrepancies.csv...
genre_discrepancies.csv guardado: 19 géneros
Top 5 géneros más valorados por usuarios:
     genre_name  discrepancy_score  game_count
12       Puzzle              -0.09          14
3   Board Games              -0.29          20
2        Arcade              -0.44          78
11   Platformer              -0.57          38
7        Family              -0.76           9


### DEVELOPER_DISCREPANCIES

In [6]:
print("\nGenerando developer_discrepancies.csv...")

# Calcular estadísticas por desarrollador
dev_stats = df_analysis.groupby('developer_name').agg({
    'metacritic': 'mean',
    'rating': 'mean',
    'discrepancy_score': 'mean',
    'id': 'count',
    'name': lambda x: list(x)  # Lista de juegos
}).round(2)

dev_stats.columns = ['metacritic_avg', 'rating_avg', 'discrepancy_score', 'game_count', 'games_list']

# Filtrar desarrolladores con al menos 0 juegos (al final me quedo también con los nichos)
dev_stats = dev_stats[dev_stats['game_count'] >= 0].reset_index()

# Encontrar el mayor hit y mayor miss de cada desarrollador
def get_extremes(dev_name, games_df):
    dev_games = games_df[games_df['developer_name'] == dev_name]
    if len(dev_games) == 0:
        return None, None
    
    biggest_hit = dev_games.loc[dev_games['discrepancy_score'].idxmax(), 'name']
    biggest_miss = dev_games.loc[dev_games['discrepancy_score'].idxmin(), 'name']
    return biggest_hit, biggest_miss

dev_stats[['biggest_hit', 'biggest_miss']] = dev_stats['developer_name'].apply(
    lambda x: pd.Series(get_extremes(x, df_analysis))
)

# Seleccionar top desarrolladores (por cantidad y extremos)
dev_discrepancies = dev_stats.sort_values('discrepancy_score', ascending=False).head(100)

# Limpiar columnas
dev_discrepancies = dev_discrepancies[['developer_name', 'metacritic_avg', 'rating_avg', 
                                     'discrepancy_score', 'game_count', 'biggest_hit', 'biggest_miss']]

# Guardar
dev_discrepancies.to_csv('../../data/processed/developer_discrepancies.csv', index=False)
print(f"developer_discrepancies.csv guardado: {len(dev_discrepancies)} desarrolladores")
print("Top 5 desarrolladores más valorados por usuarios:")
print(dev_discrepancies[['developer_name', 'discrepancy_score', 'game_count']].head())


Generando developer_discrepancies.csv...
developer_discrepancies.csv guardado: 100 desarrolladores
Top 5 desarrolladores más valorados por usuarios:
           developer_name  discrepancy_score  game_count
787    Hothouse Creations               4.92           1
908                 KEMCO               4.05           1
1970             h.a.n.d.               3.25           1
210   Big Ape Productions               2.88           1
297          CDV Software               2.80           1


### TEMPORAL_DISCREPANCIES

In [10]:
print("Generando temporal_discrepancies.csv (por año y género)...")

# Agrupar por año Y género
temporal_stats = df_analysis.groupby(['year', 'genre_name']).agg({
    'metacritic': 'mean',
    'rating': 'mean',
    'discrepancy_score': 'mean',
    'id': 'count'
}).round(2)

temporal_stats.columns = ['metacritic_avg', 'rating_avg', 'discrepancy_score', 'game_count']
temporal_stats = temporal_stats.reset_index()

# Filtrar combinaciones año-género con al menos 5 juegos
temporal_discrepancies = temporal_stats[temporal_stats['game_count'] >= 0]

# Ordenar por año y luego por discrepancia
temporal_discrepancies = temporal_discrepancies.sort_values(['year', 'discrepancy_score'], ascending=[True, False])

# Guardar
temporal_discrepancies.to_csv('../../data/processed/temporal_discrepancies.csv', index=False)
print(f"temporal_discrepancies.csv guardado: {len(temporal_discrepancies)} combinaciones año-género")

# Mostrar algunos ejemplos interesantes
print("\nEvolución por década y género (ejemplos):")
for decade in [2000, 2010, 2020]:
    decade_data = temporal_discrepancies[
        (temporal_discrepancies['year'] >= decade) & 
        (temporal_discrepancies['year'] < decade + 10)
    ]
    if len(decade_data) > 0:
        print(f"\nDécada {decade}s - Top géneros más valorados por usuarios:")
        top_decade = decade_data.nlargest(3, 'discrepancy_score')[['year', 'genre_name', 'discrepancy_score', 'game_count']]
        for _, row in top_decade.iterrows():
            print(f"  {row['year']} - {row['genre_name']}: {row['discrepancy_score']:.2f} ({row['game_count']} juegos)")

Generando temporal_discrepancies.csv (por año y género)...
temporal_discrepancies.csv guardado: 365 combinaciones año-género

Evolución por década y género (ejemplos):

Década 2000s - Top géneros más valorados por usuarios:
  2004 - Arcade: 1.88 (3 juegos)
  2002 - Arcade: 1.40 (1 juegos)
  2007 - Fighting: 1.23 (1 juegos)

Década 2010s - Top géneros más valorados por usuarios:
  2013 - Puzzle: 1.35 (2 juegos)
  2015 - Casual: 0.85 (1 juegos)
  2013 - Board Games: 0.51 (2 juegos)

Década 2020s - Top géneros más valorados por usuarios:
  2022 - Shooter: 1.20 (1 juegos)
  2020 - Puzzle: 0.52 (1 juegos)
  2024 - RPG: -0.05 (1 juegos)


### EXTREME_CASES

In [8]:
print("Generando extreme_cases.csv...")

extreme_cases_list = []

# Top casos de cada categoría
categories = ['overhyped', 'polarizing', 'consensus', 'hidden_gem']

for category in categories:
    category_data = df_analysis[df_analysis['discrepancy_type'] == category]
    
    if len(category_data) > 0:
        if category == 'overhyped':
            # Para overhyped: los más negativos (peores casos)
            top_cases = category_data.nsmallest(5, 'discrepancy_score')
        elif category == 'hidden_gem':
            # Para hidden gems: los más positivos (mejores casos)
            top_cases = category_data.nlargest(5, 'discrepancy_score')
        else:
            # Para consensus y polarizing: casos más representativos
            top_cases = category_data.sample(min(5, len(category_data)), random_state=42)
        
        for _, row in top_cases.iterrows():
            extreme_cases_list.append({
                'game_name': row['name'],
                'genre_name': row['genre_name'],
                'developer_name': row['developer_name'],
                'year': int(row['year']),
                'metacritic': int(row['metacritic']),
                'rating': round(row['rating'], 1),
                'discrepancy_score': round(row['discrepancy_score'], 2),
                'type': category,
                'reason': f"{category}_case"  # Placeholder, se puede mejorar
            })

# Crear DataFrame
extreme_cases = pd.DataFrame(extreme_cases_list)

# Guardar
extreme_cases.to_csv('../../data/processed/extreme_cases.csv', index=False)
print(f"extreme_cases.csv guardado: {len(extreme_cases)} casos extremos")

# Mostrar resumen por categoría
print("\nCasos extremos por categoría:")
for category in categories:
    cat_data = extreme_cases[extreme_cases['type'] == category]
    if len(cat_data) > 0:
        print(f"\n{category.upper()} ({len(cat_data)} casos):")
        for _, row in cat_data.head(3).iterrows():
            print(f"  {row['game_name']} ({row['year']}): Meta={row['metacritic']}, User={row['rating']}, Disc={row['discrepancy_score']}")

Generando extreme_cases.csv...
extreme_cases.csv guardado: 20 casos extremos

Casos extremos por categoría:

OVERHYPED (5 casos):
  The Corporate Machine (2015): Meta=80, User=1.0, Disc=-8.0
  Tadpole Treble (2016): Meta=90, User=1.8, Disc=-6.92
  Guncraft (2013): Meta=73, User=1.3, Disc=-6.47

POLARIZING (5 casos):
  Chronos: Before the Ashes (2020): Meta=68, User=2.9, Disc=-1.95
  Aegis Defenders (2018): Meta=76, User=2.5, Disc=-3.78
  Serious Sam: Kamikaze Attack! (2011): Meta=67, User=2.5, Disc=-3.05

CONSENSUS (5 casos):
  Kyn (2015): Meta=58, User=3.0, Disc=-0.8
  Fire Emblem: The Sacred Stones (2004): Meta=85, User=4.3, Disc=-0.35
  Captain Toad: Treasure Tracker (2014): Meta=81, User=4.1, Disc=-0.35

HIDDEN_GEM (5 casos):
  Crime Life: Gang Wars (2005): Meta=30, User=4.2, Disc=4.92
  Myst (1993): Meta=23, User=3.9, Disc=4.92
  Batman Beyond: Return of the Joker (2000): Meta=24, User=3.6, Disc=4.05


### SCATTER_DATA 

In [9]:
scatter_data = df_analysis.copy()

# Limpiar y preparar columnas para visualización
scatter_final = scatter_data[[
    'id', 'name', 'genre_name', 'developer_name', 'year',
    'metacritic', 'rating', 'consensus_score', 'discrepancy_score', 
    'discrepancy_type', 'playtime', 'added'
]].copy()

# Limpiar datos para mejor visualización
scatter_final['developer_name'] = scatter_final['developer_name'].fillna('Unknown')
scatter_final['playtime'] = scatter_final['playtime'].fillna(0)

# Añadir columnas útiles para filtros y colores
scatter_final['decade'] = (scatter_final['year'] // 10) * 10
scatter_final['metacritic_range'] = pd.cut(
    scatter_final['metacritic'], 
    bins=[0, 50, 70, 80, 90, 100], 
    labels=['Poor', 'Mixed', 'Good', 'Great', 'Universal']
)
scatter_final['rating_range'] = pd.cut(
    scatter_final['rating'], 
    bins=[0, 2, 3, 4, 5], 
    labels=['Poor', 'Mixed', 'Good', 'Excellent']
)

# Añadir flag para casos extremos (útil para highlighting)
scatter_final['is_extreme'] = scatter_final['discrepancy_type'].isin(['overhyped', 'hidden_gem'])

# Guardar dataset completo
scatter_final.to_csv('../../data/processed/scatter_data.csv', index=False)
print(f"scatter_data.csv guardado: {len(scatter_final):,} juegos (dataset completo)")

# Estadísticas del dataset final
print(f"\nESTADÍSTICAS DEL DATASET PARA VISUALIZACIÓN:")
print(f"Rango discrepancias: {scatter_final['discrepancy_score'].min():.2f} a {scatter_final['discrepancy_score'].max():.2f}")
print(f"Rango temporal: {scatter_final['year'].min()} - {scatter_final['year'].max()}")
print(f"Géneros únicos: {scatter_final['genre_name'].nunique()}")
print(f"Desarrolladores únicos: {scatter_final['developer_name'].nunique()}")

print(f"\nDistribución por década:")
print(scatter_final['decade'].value_counts().sort_index())

print(f"\nDistribución por tipo de discrepancia:")
print(scatter_final['discrepancy_type'].value_counts())

print(f"\nTop 10 géneros:")
print(scatter_final['genre_name'].value_counts().head(10))

scatter_data.csv guardado: 5,801 juegos (dataset completo)

ESTADÍSTICAS DEL DATASET PARA VISUALIZACIÓN:
Rango discrepancias: -8.00 a 4.92
Rango temporal: 1980 - 2024
Géneros únicos: 19
Desarrolladores únicos: 1995

Distribución por década:
decade
1980       9
1990     185
2000    1586
2010    3409
2020     612
Name: count, dtype: int64

Distribución por tipo de discrepancia:
discrepancy_type
consensus     3343
polarizing    1682
hidden_gem     716
overhyped       60
Name: count, dtype: int64

Top 10 géneros:
genre_name
Action        3282
Adventure      787
Strategy       505
RPG            413
Sports         169
Racing         155
Simulation     141
Arcade          78
Shooter         70
Casual          47
Name: count, dtype: int64
